In [1]:
using Printf, SparseArrays, LinearAlgebra
using IterativeSolvers, BenchmarkTools, Statistics
using PyPlot
using AbaqusReader, PyCall
using DelimitedFiles, ProgressMeter, MAT

# import LinearAlgebra.svd

In [2]:
include("adiff.jl")
include("materials.jl")
include("elements.jl")
include("autodiff_truss.jl")
;

┌ Info: Recompiling stale cache file /home/andrea/.julia/compiled/v1.1/StatsBase/EZjIG.ji for StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91]
└ @ Base loading.jl:1184


In [3]:
function calcFiKt(u, nodes, beams)
  
  N = length(u)
  ϕ, r, Kt = 0, spzeros(N), spzeros(N,N)
  
  for beam in beams
    idxs          = LinearIndices(u)[beam[:],:]    
    ϕFiKt         = calcFiKt(nodes[idxs], u[idxs])
    idxs          = idxs[:]
    ϕ             += ϕFiKt[1]
    r[idxs]       += ϕFiKt[2]
    Kt[idxs,idxs] += ϕFiKt[3]
  end
  (ϕ, r, Kt)  
end

function calcFiKt(nodes, u)
  
  C        =  diagm(0=>ones(6))
  C[2:14:30] .= C[7:14:35] .= -1

  l0       = norm(nodes[2,:]-nodes[1,:])
  Δr       = nodes[2,:]+u[2,:] - (nodes[1,:]+u[1,:])
  l        = norm(Δr)  
  Δx,Δy,Δz = Δr[:]
  dphidl   = (l/l0-1)/l0
  dphi2dl2 = 1/l0^2  
  dldu     = [-Δx, Δx, -Δy, Δy, -Δz, Δz]/l;
  fact1    = -dphidl/l^2 + dphi2dl2/l;
  
  ϕ        = (1/2) *(l/l0-1)^2
  r        = dphidl * dldu;
  Kt       = fact1*l*(dldu*transpose(dldu)) + (dphidl/l)*C

  (ϕ, r, Kt)
end

calcFiKt (generic function with 2 methods)

In [4]:
function getϕD2(u, nodes, beams)
  
  N = length(u)
  ϕ, r, C = 0, spzeros(N), spzeros(N,N)
  
  for beam in beams
    idxs         = LinearIndices(u)[beam[:],:]    
    ϕEl          = getϕ(nodes[idxs], adiff.D2(u[idxs]))
    idxs         = idxs[:]
    ϕ            += adiff.val(ϕEl)
    r[idxs]      += adiff.grad(ϕEl)
    C[idxs,idxs] += adiff.hess(ϕEl)
  end  
  (ϕ, r, C) 
end

function getϕ(nodes, u)
  
  l₀ = norm(nodes[1,:] - nodes[2,:])
  l  = norm((nodes[1,:]+u[1,:])-(nodes[2,:]+u[2,:]))
  ϕ  = 1/2*(l/l₀-1)^2  
end
;

In [5]:
function findu!(u, nodes, beams, idxf;
    bverb   = false, 
    dTol    = 1e-6,
    maxiter = 30, 
    bdual   = true)
  
  nDoFs   = length(idxf)
  updt    = zeros(nDoFs)
  iter    = 0
  bdone   = false
  bfailed = false
  
  if bverb
    p = ProgressThresh(dTol, "resudual norm:")
  end
  while !bdone && !bfailed
    global ϕ, r, C
    if bdual
      (ϕ, r, C)   = getϕD2(u, nodes, beams)
    else
      (ϕ, r, C)   = calcFiKt(u, nodes, beams)
    end
      
    res         = r[idxf]
    normr       = norm(res)/nDoFs
    bverb && update!(p, normr)
    if normr ≤ dTol
       bdone = true
    elseif iter ≤ maxiter
        minres!(updt, C[idxf,idxf], res)
        u[idxf] -= updt
        iter += 1
    else
      println("failed to converge!")
      bfailed = true
    end
  end
  return ((ϕ, r, C), !bfailed)
end

findu! (generic function with 1 method)

In [6]:
L0    = sqrt(2)/2
nodes_RVE = [[0, 0, -sqrt(2)/2],
    [0, 0, sqrt(2)/2],
    [-1/2, -1/2, 0],
    [1/2, -1/2, 0],
    [1/2, 1/2, 0],
    [-1/2, 1/2, 0],
    [0, -1, sqrt(2)/2],
    [1, 0, sqrt(2)/2],
    [0, 1, sqrt(2)/2],
    [-1, 0, sqrt(2)/2],
    [0, -1, -sqrt(2)/2],
    [1, 0, -sqrt(2)/2],
    [0, 1, -sqrt(2)/2],
    [-1, 0, -sqrt(2)/2]] * L0
beams_RVE = [[9 5], [7 3], [8 4], [10 3], 
  [7 4], [9 6], [8 5], [10 6],
  [1 3], [1 4], [1 5], [1 6], [11 3],
  [13 5], [2 3], [2 4], [2 5], [2 6],
  [3 4], [4 5], [5 6], [6 3], [7 2], [8 2],
  [9 2], [10 2], [11 1],
  [12 1], [12 4], [12 5], [13 1], [13 6],
  [14 1], [14 3], [11 4], [14 6]]
R      = zeros(3,3)
R[1,:] = [sqrt(2)/2, sqrt(2)/2, 0]
R[2,:] = [sqrt(2)/2, -sqrt(2)/2, 0]
R[3,:] = [0, 0, 1];
nodes_RVE  = [R*v for v in nodes_RVE]

;